In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import manifold

# Create 3D Gaussian

In [2]:
dim = 3
n_gauss = 2
n_pts_per_gauss = 300
np.random.seed(5)

# centers = np.zeros((n_gauss,dim))
# for i in range(1,n_gauss):
#     centers[i] = np.random.randint(0,2,3)
centers = np.random.uniform(-1,1,size=(n_gauss,3))
    
print(centers)

cov_m = [np.diag([0.01 for i in range(dim)]),np.diag([0.01 if i%2 !=0 else 0.01 for i in range(dim)])]

D = np.zeros((n_pts_per_gauss*n_gauss,dim))
c = np.zeros(n_pts_per_gauss*n_gauss)      # storage for labels
for i in range(n_gauss):
    k = np.random.randint(0,2,1)[0]
    D[i*n_pts_per_gauss:(i+1)*n_pts_per_gauss] = np.random.multivariate_normal(centers[i],cov_m[k],n_pts_per_gauss)
    c[i*n_pts_per_gauss:(i+1)*n_pts_per_gauss] = i 
D = (D-np.min(D,axis=0))/(np.max(D,axis=0)-np.min(D,axis=0))
print(D.shape)
print(c.shape)

%matplotlib qt

# colors = ['r', 'g', 'b']  # Red, Green, Blue
colors = ['#FF0000', '#00FF00', '#0000FF', '#FFFF00', '#00FFFF', '#FF00FF']
# Create a figure and 3D axis
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(projection='3d')

# Define colors for each Gaussian distribution

# Loop through each Gaussian to plot points with corresponding color
for i in range(n_gauss):
    ax.scatter(D[c == i, 0], D[c == i, 1], D[c == i, 2], color=colors[i], label=f'Gaussian {i+1}')
    # ax.scatter(D[:,0], D[:,1], D[:,2], c=c)

# Set labels and title
ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_zlabel('Z-axis')
ax.set_title('3D Scatter Plot of Data Points from Three Gaussian Distributions')

# Add a legend
ax.legend()

# Show the plot
plt.show()

[[-0.55601366  0.74146461 -0.58656169]
 [ 0.83722182 -0.02317762  0.22348773]]
(600, 3)
(600,)


# Project 3D to 2D

In [3]:
t_sne = manifold.TSNE(
    n_components=2,
    perplexity=30,
    init="random",
    random_state=0,
)

S = t_sne.fit_transform(D)

# Plotting the t-SNE results with the same color scheme
%matplotlib qt

# colors = ['r', 'g', 'b','']  # Red, Green, Blue
plt.figure(figsize=(10, 8))
for i in range(n_gauss):
    plt.scatter(S[c == i, 0], S[c == i, 1], color=colors[i], label=f'Gaussian {i+1}')
    # plt.scatter(S[c == i, 0], S[c == i, 1], label=f'Gaussian {i+1}')

plt.title('t-SNE Visualization of 3D Gaussian Distributions into 2D')
plt.legend()
plt.grid(True)
plt.show()

## Define the Inverse Projection

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import train_test_split

# Define the MLP inverse_model
class NNinv(nn.Module):
    def __init__(self, input_size, output_size):
        super(NNinv, self).__init__()
        
        # Define the layers
        self.layers = nn.Sequential(
            nn.Linear(input_size, 64),  # Input to first hidden layer
            nn.ReLU(),
            nn.Linear(64, 128),  # First hidden layer to second hidden layer
            nn.ReLU(),
            nn.Linear(128, 256),  # Second hidden layer to third hidden layer
            nn.ReLU(),
            nn.Linear(256, 512),  # Third hidden layer to fourth hidden layer
            nn.ReLU(),
            nn.Linear(512, output_size),  # Fifth hidden layer to output
            nn.Sigmoid()  # Output layer with sigmoid activation
        )
    
    def forward(self, x):
        return self.layers(x)
X_train, X_test, y_train, y_test, c_train, c_test = train_test_split(S, D,c, test_size=0.33, random_state=42, stratify=c)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(c_train.shape)
print(c_test.shape)

(402, 2)
(198, 2)
(402, 3)
(198, 3)
(402,)
(198,)


In [5]:
# Example usage
input_size = 2  # Example input size (can be changed)
output_size = dim   # Binary classification (sigmoid output for single output)

# Create DataLoader for batch processing
batch_size = 64
t_X_train = torch.tensor(X_train)
t_y_train = torch.tensor(y_train)
dataset = TensorDataset(t_X_train, t_y_train)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Instantiate the inverse_model, loss function, and optimizer
inverse_model = NNinv(input_size, output_size)
loss_fn = nn.L1Loss()  # Mean Absolute Error (MAE)
optimizer = optim.Adam(inverse_model.parameters(), lr=0.001)

# Number of epochs to train
num_epochs = 5

# Training loop
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs, targets) in enumerate(dataloader):
        # Forward pass
        outputs = inverse_model(inputs)
        loss = loss_fn(outputs, targets)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    # Print the average loss for the epoch
    avg_loss = running_loss / len(dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

print("Training complete.")

t_X_test = torch.tensor(X_test)
t_y_test = torch.tensor(y_test)
outputs_test = inverse_model(t_X_test)
loss_test = loss_fn(outputs_test, t_y_test)
print(loss_test/y_test.shape[0])

%matplotlib qt

# Create a figure and 3D axis
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(projection='3d')

# Define colors for each Gaussian distribution
# colors = ['r', 'g', 'b']  # Red, Green, Blue


output_fin = outputs_test.detach().numpy()
# Loop through each Gaussian to plot points with corresponding color
for i in range(n_gauss):
    ax.scatter(t_y_test[c_test == i, 0], t_y_test[c_test == i, 1], t_y_test[c_test == i, 2], color=colors[i], label=f'Actual_Gaussian {i+1}')
    # ax.scatter(output_fin[c_test == i, 0], output_fin[c_test == i, 1], output_fin[c_test == i, 2], color='orange', label=f'Predicted_Gaussian {i+1}')

ax.scatter(output_fin[:, 0], output_fin[:, 1], output_fin[:, 2], color='orange', label=f'Predicted_Gaussians')

# Set labels and title
ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_zlabel('Z-axis')
ax.set_title('TSNE \n Actual Vs Prediction')

# Add a legend
ax.legend()

# Show the plot
plt.show()

Epoch [1/5], Loss: 0.1382
Epoch [2/5], Loss: 0.0901
Epoch [3/5], Loss: 0.0779
Epoch [4/5], Loss: 0.0728
Epoch [5/5], Loss: 0.0692
Training complete.
tensor(0.0004, dtype=torch.float64, grad_fn=<DivBackward0>)


## Create a 2D Grid for Jacobian Calculation

In [6]:
# Define min and max values
x_min, x_max = np.min(S[:, 0]), np.max(S[:, 0])
y_min, y_max = np.min(S[:, 1]), np.max(S[:, 1])
print(x_min, x_max)
print(y_min, y_max)

# Define grid resolution
num_grid_points = 100

# Generate grid
x_vals = np.linspace(x_min, x_max, num_grid_points)
y_vals = np.linspace(y_min, y_max, num_grid_points)
xx, yy = np.meshgrid(x_vals, y_vals)
print(yy.shape)
print(y_vals.shape)

-35.52948 32.871674
-25.627789 26.346518
(100, 100)
(100,)


In [7]:
%matplotlib qt

plt.figure(figsize=(10, 8))
# Visualize the grid on top of the t-SNE data
plt.scatter(S[:, 0], S[:, 1], c='blue', s=10, label="t-SNE Output")
plt.scatter(xx, yy, c='red', s=5, label="Grid Points")
plt.title("2D t-SNE Output with Grid Points")
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
# plt.grid(True)
plt.show()

## Define Jacobian

In [17]:
def compute_jacobian(x, y):
    """
    Computes the Jacobian matrix for each point in the grid.
    
    Args:
        grid_points (ndarray): A 2D array of shape (n_points, 2) representing the grid points.

    Returns:
        jacobian_matrices (list): A list of jacobian matrices for each grid point.
    """
    jacobian_matrices = []
    
    # Define the model's forward pass to use autograd
    def model_forward(input):
        return inverse_model(input)  # Model's forward pass
    
    # Iterate through the grid points
    # for point in grid_points:
    point_tensor = torch.tensor([x, y], dtype=torch.float32, requires_grad=True)  # (1, 2) tensor
    
    # Compute the Jacobian using autograd's jacobian function
    jacobian = torch.autograd.functional.jacobian(model_forward, point_tensor)
    
        # The output of jacobian will have shape (1, 3, 2), so we need to squeeze to get (3, 2)
        # jacobian_matrices.append(jacobian.squeeze(0))  # Remove the batch dimension
    
    return jacobian

# ### Function to compute Jacobian at a specific point
# def compute_jacobian_implement(x, y, eps):
#     # eps = 1e-5  # Small epsilon for numerical differentiation

#     # Partial derivatives with respect to x 
#     point_hor_plus = torch.tensor([[x + eps, y]]) 
#     point_hor_minus = torch.tensor([[x - eps, y]]) 
#     f_x_plus_eps = inverse_model(point_hor_plus).detach().numpy()   #3D output
#     f_x_minus_eps = inverse_model(point_hor_minus).detach().numpy()
#     df_dx = (f_x_plus_eps - f_x_minus_eps) / (2 * eps)

#     # Partial derivatives with respect to y
#     point_ver_plus = torch.tensor([[x , y + eps]]) 
#     point_ver_minus = torch.tensor([[x , y - eps]]) 
#     f_y_plus_eps = inverse_model(point_ver_plus).detach().numpy()
#     f_y_minus_eps = inverse_model(point_ver_minus).detach().numpy()
#     df_dy = (f_y_plus_eps - f_y_minus_eps) / (2 * eps)

#     # Jacobian matrix 3x2
#     J = np.column_stack((df_dx.T, df_dy.T))
#     return J
###################################################################################################
# def compute_jacobian_implement(x, y, eps):
#     # Create tensors for the input points
#     point_hor_plus = torch.tensor([[x + eps, y]], dtype=torch.float32) 
#     point_hor_minus = torch.tensor([[x - eps, y]], dtype=torch.float32)
#     point_ver_plus = torch.tensor([[x, y + eps]], dtype=torch.float32)
#     point_ver_minus = torch.tensor([[x, y - eps]], dtype=torch.float32)
    
#     # Evaluate function at shifted points
#     f_x_plus_eps = inverse_model(point_hor_plus).detach().numpy().squeeze()  # Ensure 1D
#     f_x_minus_eps = inverse_model(point_hor_minus).detach().numpy().squeeze()
#     f_y_plus_eps = inverse_model(point_ver_plus).detach().numpy().squeeze()
#     f_y_minus_eps = inverse_model(point_ver_minus).detach().numpy().squeeze()
    
#     # Compute partial derivatives using finite differences
#     df_dx = (f_x_plus_eps - f_x_minus_eps) / (2 * eps)
#     df_dy = (f_y_plus_eps - f_y_minus_eps) / (2 * eps)
    
#     # Construct the Jacobian (3x2 matrix)
#     J = np.column_stack((df_dx, df_dy))
#     return J


def compute_jacobian_implement(x, y, eps=1e-5):
    # Create tensor point for cloning
    point = torch.tensor([[x, y]], dtype=torch.float32)

    # Partial derivative w.r.t. x using five-point stencil
    f_x_2plus = inverse_model(torch.tensor([[x + 2 * eps, y]], dtype=torch.float32))
    f_x_plus = inverse_model(torch.tensor([[x + eps, y]], dtype=torch.float32))
    f_x_minus = inverse_model(torch.tensor([[x - eps, y]], dtype=torch.float32))
    f_x_2minus = inverse_model(torch.tensor([[x - 2 * eps, y]], dtype=torch.float32))
    
    df_dx = (-f_x_2plus + 8 * f_x_plus - 8 * f_x_minus + f_x_2minus) / (12 * eps)

    # Partial derivative w.r.t. y using five-point stencil
    f_y_2plus = inverse_model(torch.tensor([[x, y + 2 * eps]], dtype=torch.float32))
    f_y_plus = inverse_model(torch.tensor([[x, y + eps]], dtype=torch.float32))
    f_y_minus = inverse_model(torch.tensor([[x, y - eps]], dtype=torch.float32))
    f_y_2minus = inverse_model(torch.tensor([[x, y - 2 * eps]], dtype=torch.float32))
    
    df_dy = (-f_y_2plus + 8 * f_y_plus - 8 * f_y_minus + f_y_2minus) / (12 * eps)

    # Stack results to form Jacobian matrix
    jacobian = torch.stack([df_dx.squeeze(), df_dy.squeeze()], dim=1)
    
    return jacobian.detach().numpy()


In [18]:
# Calculate Jacobians over the grid and store results
jacobians = []
for i in range(num_grid_points):
    for j in range(num_grid_points):
        x, y = xx[i, j], yy[i, j]
        # print(x,y)
        jacobian_mt = compute_jacobian_implement(x, y, 1e-5)
        # jacobian_mt = compute_jacobian(x, y)
        # print(jacobian)
        jacobians.append(jacobian_mt)

# Reshaping jacobina

In [13]:
# jacobians[0]

In [19]:
jacobians[0]

array([[ 0.02483527, -0.03476938],
       [-0.01564622,  0.01316269],
       [ 0.02135833, -0.02880891]], dtype=float32)

In [20]:
# Convert the list of numpy arrays into a list of PyTorch tensors
jacobian_tensors = [torch.tensor(jacob) for jacob in jacobians]

# Convert the list into a 3D tensor
jacobian_tensor = torch.stack(jacobian_tensors)  # Shape will be [num_grids * num_grids, 3, 2]

# Reshape the tensor to [num_grids, num_grids, 3, 2]
jacobian_tensor_reshaped = jacobian_tensor.view(num_grid_points, num_grid_points, 3, 2)
jacobian_tensor_reshaped.shape

torch.Size([100, 100, 3, 2])

# Eigne_value

In [ ]:
# Compute the eigenvalues of J^T J for each Jacobian matrix
eigenvalues = np.zeros((num_grid_points, num_grid_points, 2))  # Store two eigenvalues per J^T J (2x2 matrix)

for i in range(num_grid_points):
    for j in range(num_grid_points):
        jacobian = jacobian_tensor_reshaped[i, j]  # Get the 3x2 Jacobian matrix
        gram_matrix = jacobian.T @ jacobian  # Compute the 2x2 J^T J matrix
        eigvals = torch.linalg.eigvals(gram_matrix)  # Eigenvalues of J^T J
        eigenvalues[i, j] = eigvals.real.numpy()  # Store only real parts if complex

# Visualize the largest eigenvalue across the grid to identify sensitive areas
largest_eigenvalue = np.max(eigenvalues, axis=2)
# sqrt_large_eig_val = np.sqrt(largest_eigenvalue)

plt.figure(figsize=(8, 6))
plt.imshow(largest_eigenvalue, cmap="hot")
# plt.imshow(sqrt_large_eig_val, cmap="hot")
plt.colorbar(label="Largest Eigenvalue of J^T J (Sensitivity)")
plt.xlabel("X-axis")
plt.ylabel("Y-axis")
plt.title("Heatmap of Largest Eigenvalue (Sensitive Regions)")
plt.show()

: 

# Numerically & Analytically Testing

In [16]:
import torch
import numpy as np

# Define the analytic function f
def analytic_function(point):
    x, y = point[:, 0], point[:, 1]
    z1 = torch.sin(x) + y
    z2 = torch.cos(y) - x
    z3 = x**2 + y**2
    return torch.stack([z1, z2, z3], dim=1)

# Define symbolic Jacobian for testing
def symbolic_jacobian(x, y):
    J = np.array([
        [np.cos(x), 1],
        [-1, -np.sin(y)],
        [2 * x, 2 * y]
    ])
    return J

# Numerical Jacobian function
def compute_jacobian_autograd_single_point(func, point):
    point_tensor = torch.tensor([point], dtype=torch.float32, requires_grad=True)
    jacobian = torch.autograd.functional.jacobian(func, point_tensor).squeeze(0)
    return jacobian.detach().numpy()

def compute_jacobian_implement_analytic(x, y, eps=1e-5):
    # Create tensor point for cloning
    point = torch.tensor([[x, y]], dtype=torch.float32)

    # Partial derivative w.r.t. x using five-point stencil
    f_x_2plus = analytic_function(torch.tensor([[x + 2 * eps, y]], dtype=torch.float32))
    f_x_plus = analytic_function(torch.tensor([[x + eps, y]], dtype=torch.float32))
    f_x_minus = analytic_function(torch.tensor([[x - eps, y]], dtype=torch.float32))
    f_x_2minus = analytic_function(torch.tensor([[x - 2 * eps, y]], dtype=torch.float32))
    
    df_dx = (-f_x_2plus + 8 * f_x_plus - 8 * f_x_minus + f_x_2minus) / (12 * eps)

    # Partial derivative w.r.t. y using five-point stencil
    f_y_2plus = analytic_function(torch.tensor([[x, y + 2 * eps]], dtype=torch.float32))
    f_y_plus = analytic_function(torch.tensor([[x, y + eps]], dtype=torch.float32))
    f_y_minus = analytic_function(torch.tensor([[x, y - eps]], dtype=torch.float32))
    f_y_2minus = analytic_function(torch.tensor([[x, y - 2 * eps]], dtype=torch.float32))
    
    df_dy = (-f_y_2plus + 8 * f_y_plus - 8 * f_y_minus + f_y_2minus) / (12 * eps)

    # Stack results to form Jacobian matrix
    jacobian = torch.stack([df_dx.squeeze(), df_dy.squeeze()], dim=1)
    
    return jacobian.detach().numpy()

# Define a grid point for testing
x = 0.5
y = -0.5
test_point = np.array([x, y])

# Compute the symbolic Jacobian
symbolic_J = symbolic_jacobian(test_point[0], test_point[1])

# Compute the numerical Jacobian using autograd
numerical_J = compute_jacobian_autograd_single_point(analytic_function, test_point)

implent_j = compute_jacobian_implement_analytic(x, y, 1e-5)

# Compare the results
print("Symbolic Jacobian:\n", symbolic_J)
print("Numerical Jacobian:\n", numerical_J)
print('Implementation', implent_j)
print("Difference:\n", implent_j - symbolic_J)


Symbolic Jacobian:
 [[ 0.87758256  1.        ]
 [-1.          0.47942554]
 [ 1.         -1.        ]]
Numerical Jacobian:
 [[[ 0.87758255  1.        ]]

 [[-1.          0.47942555]]

 [[ 1.         -1.        ]]]
Implementation [[ 0.87966526  1.0016065 ]
 [-1.0016065   0.4803141 ]
 [ 1.0016065  -1.001358  ]]
Difference:
 [[ 0.00208269  0.00160646]
 [-0.00160646  0.00088857]
 [ 0.00160646 -0.00135803]]
